### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression

def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation) #parametry tej funkcji: x – znaki do zamiany.
    return text.translate(translator)                      #y – znaki, na które mają być zamienione znaki z x.
                                                           #z – znaki do usunięcia.
baby_df = pd.read_csv('amazon_baby.csv')
baby_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [4]:
#a)

#short test:
baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'
remove_punctuation(baby_df["review"][4]) == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

True

In [5]:
baby_df["review"] = baby_df["review"].fillna("") #zamieniamy nan na pusty string
baby_df["review"][38] == baby_df["review"][38]   #(nie mozna uzyc dropna bo to by usunelo cale wiersze wylacznie z NAN gdzie jest nan jako review)

True

In [6]:
#c)

baby_df = baby_df[baby_df["rating"] != 3]
sum(baby_df["rating"] == 3)



0

In [7]:

def gorsze_rozwiazanie(baza):
  baza.loc[baza['rating']==2,'rating'] = -1
  baza.loc[baza['rating']==1,'rating'] = -1
  baza.loc[baza['rating']==4,'rating'] = 1
  baza.loc[baza['rating']==5,'rating'] = 1
  return baza

def lepsze_rozwiazanie(baza):
    baza.loc[baza['rating'] <= 2, 'rating'] = -1
    baza.loc[baza['rating'] >= 4, 'rating'] = 1
    return baza


baby_df=lepsze_rozwiazanie(baby_df)
#baby_df=gorsze_rozwiazanie(baby_df)

sum(baby_df["rating"]**2 != 1)

0

In [ ]:
baby_df.head()

,name,review,rating
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",1


## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names())
print(X_train_example.todense())



['adore' 'and' 'apples' 'bananas' 'dislike' 'hate' 'like' 'oranges' 'they'
 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


In [9]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer.

In [10]:
#a)
from sklearn.model_selection import train_test_split
X,y=baby_df["review"],baby_df["rating"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [11]:
#b)
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)


## Exercise 3
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [12]:
#a)
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [30]:
#b)
coefficients = model.coef_[0]#tworzy wagi slow (ocenia ich wartosc) [[-1.5, 2.0, 3.0, -2.5, 1.2]]
feature_names = vectorizer.get_feature_names_out()#pobieramy nazwy tych slow po ich klasyfikacji

# Pair words with their coefficients
word_coef = zip(feature_names, coefficients)#przyrownujemy te wagi do tych slow
sorted_words = sorted(word_coef, key=lambda x: x[1]) #sortujemy po tych wspolczynnikach od najgorszych do najepszych

# 10 most negative and 10 most positive words
most_negative = sorted_words[:10] #pobieramy liste pierwszych 10 elementow czyli najgorszych
most_positive = sorted_words[-10:]#pobieramy liste ostatnich 10 elementow czyli najlepszych

print("10 Most Negative Words:")
for word, coef in most_negative:
    print(f"{word}: {coef:.4f}")

print("\n10 Most Positive Words:")
for word, coef in most_positive:
    print(f"{word}: {coef:.4f}")

10 Most Negative Words:
disappointing: -2.9343
worst: -2.8849
useless: -2.6873
poorly: -2.4657
poor: -2.3934
returning: -2.3846
terrible: -2.3654
concept: -2.2862
returned: -2.1039
disappointed: -2.0976

10 Most Positive Words:
satisfied: 1.7180
perfectly: 1.7369
perfect: 1.7435
highly: 1.8277
glad: 1.8349
amazing: 1.9862
pleased: 1.9912
awesome: 2.0274
worry: 2.0497
excellent: 2.2702


## Exercise 4
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [22]:
#a)
y_pred = model.predict(X_test_vectorized)

In [50]:
#b)
y_prob=model.predict_proba(X_test_vectorized)
positive_probs = y_prob[:, 1]
#hint: model.predict_proba()


In [54]:
#c)
X_test_df = pd.DataFrame({'review': X_test, 'predicted_sentiment': y_pred, 'positive_prob': positive_probs})
most_positive = X_test_df.nlargest(5, 'positive_prob')
most_negative = X_test_df.nsmallest(5, 'positive_prob')

print(most_positive)
print(most_negative)

                                                   review  \
74899   We love this highchair.  We have a 4 year old ...   
123632  I did a TON of research before I purchased thi...   
135152  We've been using Britax for our boy (now 14 mo...   
50735   The joovy zoom 360 was the perfect solution fo...   
100166  I bought this carrier when my daughter was abo...   

        predicted_sentiment  positive_prob  
74899                     1            1.0  
123632                    1            1.0  
135152                    1            1.0  
50735                     1            1.0  
100166                    1            1.0  
                                                   review  \
147902  My disappointment with this product prompted m...   
175191  I had to return this stroller for three reason...   
155287  This is my second video monitoring system, the...   
27310   PLEASE HEED THE OTHER REVIEWERS WARNINGS ON TH...   
57234   My husband and I are VERY disappointed and sho... 

In [58]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)

# Print results
print("Accuracy of the model:", accuracy)


Accuracy of the model: 0.9345147072051813


## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [59]:
significant_words = ['love','great','easy','old','little','perfect','loves','well','able','car','broke','less','even','waste','disappointed','work','product','money','would','return']

In [64]:
#zad2
X,y=baby_df["review"],baby_df["rating"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
vectorizer = CountVectorizer(vocabulary=significant_words)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)
#zad3
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)
words = vectorizer.get_feature_names_out()
coefficients = model.coef_.flatten()
top_positive_indices = np.argsort(coefficients)[-10:]
top_negative_indices = np.argsort(coefficients)[:10]
top_positive_words = words[top_positive_indices]
top_negative_words = words[top_negative_indices]
#zad4
y_pred = model.predict(X_test_vectorized)
y_prob = model.predict_proba(X_test_vectorized)
positive_probs = y_prob[:, 1]
positive_reviews_indices = np.argsort(y_prob)[-5:]
negative_reviews_indices = np.argsort(y_prob)[:5]
positive_reviews = X_test.iloc[positive_reviews_indices[:, 1]]
negative_reviews = X_test.iloc[negative_reviews_indices[:, 1]]
accuracy = accuracy_score(y_test, y_pred)
#most_positive = X_test_df.nlargest(5, 'positive_prob')
#most_negative = X_test_df.nsmallest(5, 'positive_prob')




In [66]:
#b)
word_impact = dict(zip(vectorizer.get_feature_names_out(), model.coef_.flatten()))
sorted_word_impact = sorted(word_impact.items(), key=lambda x: x[1], reverse=True)
for word, impact in sorted_word_impact:
    print(f"{word}: {impact:.4f}")

loves: 1.6906
perfect: 1.5118
love: 1.3495
easy: 1.1833
great: 0.9287
well: 0.5178
little: 0.4989
able: 0.1953
car: 0.0729
old: 0.0686
less: -0.1897
product: -0.3177
would: -0.3416
even: -0.4892
work: -0.6374
money: -0.9261
broke: -1.6633
waste: -1.9580
return: -2.1027
disappointed: -2.4054


In [70]:
#c)
import time
full_vectorizer = CountVectorizer()
X_train_full_vec = full_vectorizer.fit_transform(X_train)
X_test_full_vec = full_vectorizer.transform(X_test)

# Train LogisticRegression on full vocabulary
model_full = LogisticRegression()
model_full.fit(X_train_full_vec, y_train)

# Predict and calculate accuracy
y_pred_full = model_full.predict(X_test_full_vec)
accuracy_full = accuracy_score(y_test, y_pred_full)

print("Accuracy with restricted vocabulary:", accuracy)
print("Accuracy with full vocabulary:", accuracy_full)
start_time = time.time()
model.fit(X_train_vectorized, y_train)
y_pred_restricted = model.predict(X_test_vectorized)
end_time = time.time()
time_restricted = end_time - start_time

# Measure time for full vocabulary model
start_time = time.time()
model_full.fit(X_train_full_vec, y_train)
y_pred_full = model_full.predict(X_test_full_vec)
end_time = time.time()
time_full = end_time - start_time
#hint: %time, %timeit
print(f"Time for restricted vocabulary model: {time_restricted:.4f} seconds")
print(f"Time for full vocabulary model: {time_full:.4f} seconds")

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy with restricted vocabulary: 0.8693592396030104
Accuracy with full vocabulary: 0.9345147072051813
Time for restricted vocabulary model: 0.2205 seconds
Time for full vocabulary model: 12.3669 seconds


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
